# Programa Ingenias+ Data Science

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
import sklearn

In [ ]:
# Chequeamos nuestra version
sklearn.__version__

Ya dijimos previamente que un proyecto de data science tiene varias etapas:

1. Recolección de Datos
2. Exploración y Procesamiento de los datos
3. Modelado
4. Puesta en Producción

En la clase anterior, hicimos el analisis exploratorio de los datos y pudimos observar el tipo de datos que teniamos. Pudimos hacernos preguntas y ver algunos patrones.

Una vez que visualizamos y exploramos el dataset tenemos una idea de como lucen nuestros datos. Es por eso que ahora debemos empezar a preparar nuestros datos para los siguientes pasos según lo que aprendimos de ellos y las preguntas que nos planteamos.

In [ ]:
# Leemos nuevamente los datos del blackfriday
blackfriday = pd.read_csv('blackfriday.csv')

In [ ]:
blackfriday.head()

#### MANEJO DE DATOS FALTANTES 🚨

Primero, vamos a recordar si tenemos datos faltantes.

In [ ]:
blackfriday.isna().sum()

¿Cuanto representa esos valores faltantes?

In [ ]:
blackfriday.isna().sum()/blackfriday.shape[0] * 100

**¿Que debo preguntarme al manejar datos faltantes?**

- ¿Quiero conservar la información?
- ¿Qué tipo de datos tengo en la columna que voy a imputar?
- ¿Por qué puede ser que tengo los valores faltantes?:
    - Valores faltantes al azar
    - Valores faltantes no al azar
- Si es una variable numerica continua, ¿Que distribución tiene?

**¿Como decido que hago con los valores faltantes?**

- Entender que no hay una manera _perfecta_ de manejar los valores faltantes.

- Depende de mis datos

- Observar no solo la cantidad de datos faltantes sino también su patrón.

- Estrategias:  
     - **Eliminar los valores**:
        - Si la recogida de datos no se ha realizado de forma aleatoria introduce sesgo.  
        - En el caso de que haya mucho patrones con datos faltantes, podría reducir considerablemente la cantidad de filas disponibles.
     - **Eliminación de la variable (columna)**:
         - Las variables “descartadas” podrían contener información de vital importancia.      
     - **Imputar los valores**:
        - Sustituir por media o mediana: Solo en variables numericas  
        - Sustituir por valor mas frecuente: Util para variables categoricas
        - Algoritmo de ML: K-NN (k-nearest neighboors)

A) Si optamos por eliminar los datos, puedemos hacerlo usando la siguiente función:
    
`.dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)`

- **axis**: {0, 1}, default 0. 0: Filas, 1: Columnas
- **how**: {'any', 'all'}, default 'any'. Any: Si hay un valor faltante elimina la columna o fila. All: Si todos los valores de la fila o columna son faltantes, elimina la fila o columna.
- **subset**: Nombre de filas o columnas donde buscar valores faltantes
- **inplace**: True modifica el DataFrame original

[documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)

In [ ]:
blackfriday_drop = blackfriday.dropna()

In [ ]:
blackfriday_drop.shape

In [ ]:
blackfriday.dropna(how='all')

In [ ]:
blackfriday.dropna(subset=['Gender'])

In [ ]:
blackfriday.dropna(subset=['Age', 'Gender'])

B) Si optamos por imputar los datos, podemos hacerlo usando la siguiente función:
    
`sklearn.impute.SimpleImputer(missing_values=nan, strategy=’mean’, fill_value=None, copy=True, add_indicator=False)`
    
- **missing_values**: Indica como son representados los valores faltantes (np.nan es el estandard)
- **strategy**: `mean`, `median`, `most_frequent`, `constant`.
- **fill_value**: Si uso `constant`, puedo usar `fill_value` para indicar por cuál valor debe ser reemplazado
- **copy**: Si pongo True, crea una copia de X
- **add_indicator**: Agrega un indicador para los valores faltantes.


[documentación](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer)

In [ ]:
from sklearn.impute import SimpleImputer

**Variable: `City_Category`**

In [ ]:
sns.countplot(x="Occupation", data=blackfriday, palette="Set3")
sns.despine()

In [ ]:
blackfriday['Occupation'].value_counts()

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy="most_frequent")

In [ ]:
blackfriday_imputados = imputer.fit_transform(blackfriday[['Occupation']])

In [ ]:
np.shape(blackfriday_imputados)

In [ ]:
blackfriday_imputados[:10]

In [ ]:
blackfriday["Occupation"] = blackfriday_imputados

In [ ]:
blackfriday.head()

In [ ]:
blackfriday['Occupation'].value_counts()

**Variable: `Age`**

In [ ]:
blackfriday['Age'].describe()

In [ ]:
imputer_age = SimpleImputer(missing_values=np.nan, strategy="mean")

In [ ]:
blackfriday["Age"] = imputer_age.fit_transform(blackfriday[['Age']])

In [ ]:
blackfriday['Age'].describe()

**Variable: `Gender`**

In [ ]:
blackfriday['Gender'].value_counts()

In [ ]:
imputer_gen = SimpleImputer(missing_values=np.nan, strategy="most_frequent")

In [ ]:
blackfriday['Gender'] = imputer_gen.fit_transform(blackfriday[['Gender']])

In [ ]:
blackfriday['Gender'].value_counts()

**Variable `Purchase`**

In [ ]:
blackfriday.dropna(subset=['Purchase'], inplace=True)

**Columnas `Product_Category_2` and `Product_Category_3`**

In [ ]:
blackfriday.drop(['Product_Category_2', 'Product_Category_3'], axis=1, inplace=True)

In [ ]:
blackfriday.isna().sum()

#### DETECCION DE OUTLIERS

- Manera de detectar outliers:
    - z-score
        - Considera que todo valor que se aleje mucho de la media=0 es un outlier.
        - Depende de la media y el desvio estandard para medir la centralidad y dispersión, los cuales son muy afectados por outliers. Si las variables no tiene una distribución normal, termino removiendo muchos outliers del dataset.
        - No funciona bien en dataset muy pequeños.
    - Rango Intercuartil:
        - Es robusto y no es tan sensible a outliers.

In [ ]:
blackfriday.dtypes

In [ ]:
sns.displot(blackfriday['Purchase'])
sns.despine()

In [ ]:
sns.barplot(blackfriday['Purchase'], color='#F5B041')
sns.despine()

In [ ]:
z = stats.zscore(np.array(blackfriday['Purchase']))

In [ ]:
threshold = 2.5

In [ ]:
z_index = blackfriday['Purchase'][np.abs(z) < threshold].index

blackfriday_withzscore = blackfriday.loc[z_index]

In [ ]:
sns.barplot(blackfriday_withzscore['Purchase'], color='#F5B041')
sns.despine()

In [ ]:
sns.boxplot(blackfriday['Purchase'])
sns.despine()

In [ ]:
q1 = blackfriday['Purchase'].quantile(0.25)
q3 = blackfriday['Purchase'].quantile(0.75)

In [ ]:
iqr = q3 - q1
lb = q1 - (iqr * 1.5)
ub = q3 + (iqr * 1.5)

In [ ]:
blackfriday = blackfriday[(blackfriday['Purchase'] > lb) & (blackfriday['Purchase'] < ub)]

In [ ]:
blackfriday.reset_index(drop=True, inplace=True)

In [ ]:
sns.boxplot(blackfriday['Purchase'])
sns.despine()